In [ ]:
import utils
from pathlib import Path
import dataset

ROOT_DIR = Path('../')
utils.set_log_lvl('WARN')

labels = utils.load_json(ROOT_DIR / 'dataset' /
                         'Thyroid_AI_data_20220103.json')
mass_info = dataset.get_mass_info(labels)
mass_info.to_csv(ROOT_DIR / 'output' / 'mass_info.csv')

mass_info.info()

In [ ]:
from pathlib import Path
import dataset

ROOT_DIR = Path('../')
utils.set_log_lvl('WARN')

path_list = [ndpi for ndpi in (ROOT_DIR / 'dataset' / 'ndpi').glob('*.ndpi')]
slide_info = dataset.get_slide_info(path_list)
slide_info.to_csv(ROOT_DIR / 'output' / 'slide_info.csv')

slide_info.info()

In [ ]:
import openslide
import cv2
import numpy as np
from PIL import Image
import pandas as pd
from pathlib import Path
import image

ROOT_DIR = Path('../')
mass_info = pd.read_csv(ROOT_DIR / 'output' / 'mass_info.csv', index_col=0)
slide_info = pd.read_csv(ROOT_DIR / 'output' / 'slide_info.csv', index_col=0)

images = []

for _, row in mass_info.sample(64, random_state=666 ** 3).iterrows():
    slide = openslide.open_slide(
        ROOT_DIR / 'dataset' / 'ndpi' / (row['slide'] + '.ndpi'))
    img = slide.read_region(
        (row['x_min'], row['y_min']), 0, (row['width'], row['height']))
    img = image.suppress_bg(img, slide_info.loc[row['slide']])

    # gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    gray = np.array(img)[..., 1]

    # _, binary = cv2.threshold(gray, 145, 255, cv2.THRESH_BINARY)
    # contours, _ = cv2.findContours(
    #     binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # contours = list(filter(lambda x: len(x) >= 5 and len(x) <= 75, contours))
    # img_data = np.array(img)
    # cv2.drawContours(img_data, contours, -1, (255, 0, 0), 2)

    # display(Image.fromarray(img_data))

    binary = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 35, 0.5)
    contours, _ = cv2.findContours(
        binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # contours = list(filter(lambda x: len(x) >= 20 and len(x) <= 200, contours))
    img_data = np.array(img)
    img_data = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
    for idx in range(len(contours)):
        r_intensity = min(len(contours[idx]) * 2, 255)
        g_intensity = 255 - min(cv2.contourArea(contours[idx]) * 15, 255)
        cv2.drawContours(img_data, contours, idx,
                         (30, g_intensity, r_intensity), 1)
    img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
    # display(Image.fromarray(img_data))
    images.append(Image.fromarray(img_data))

display(image.grid_composite(images, 5, 300, 300))